In [2]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

import numpy as np
from scipy.spatial import cKDTree

def get_crack_surface_points(input_file):
    with open(input_file, 'r') as crack_surface_points_file:
        crack_surface_points_file.readline()
        crack_surface_points = []
        for line in crack_surface_points_file:
            tokens = line.split(',')
            x = float(tokens[3])
            y = float(tokens[4])
            z = float(tokens[5])
            crack_surface_points.append((x, y, z))
    return crack_surface_points


crack_surface_points = get_crack_surface_points('../../data/0-D1_grid_points_with_attributes_transformed.csv')
crack_surface_points = np.array(crack_surface_points)
surface_xy_pts = crack_surface_points[:,0:2]
kd_tree = cKDTree(surface_xy_pts)

In [15]:
# http://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python/13849249#13849249

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [17]:
from math import pi, sin, cos, acos, atan

# This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
reference_x = -1.9932
reference_y = 363.26
step_size = 10

with open('../../data/5-crack-surface-normal.csv', 'r') as r:
    with open('../../data/6-beta.csv', 'w') as w:
        # Write the file headers
        headers = ('dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,'
                   'nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,'
                   'vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,'
                   'magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,unit_vector_to_grain_boundary_y,'
                   'unit_vector_to_grain_boundary_z,phi1,Phi,phi2,'
                   'nearest_grain_phi1,nearest_grain_Phi,nearest_grain_phi2,'
                   'misorientation,normal_x,normal_y,normal_z,raw_angle,scaled_angle\n')
        w.write(headers)
        
        # Skip the headers of the file
        r.readline()
        
        for line in r:
            # Read in the tokens
            tokens = line.split(',')
            
            dadN = tokens[0]
            crack_id = tokens[1]
            theta = tokens[2]
            
            x = float(tokens[3])
            y = float(tokens[4])
            z = float(tokens[5])
            on_crack_front = tokens[6] == 'True'
            grain_id = int(tokens[7])
            
            nearest_grain_boundary_x = tokens[8]
            nearest_grain_boundary_y = tokens[9]
            nearest_grain_boundary_z = tokens[10]
            nearest_grain_boundary_id = int(tokens[11])
            
            distance_to_grain_boundary = tokens[12]
            vector_to_grain_boundary_x = float(tokens[13])
            vector_to_grain_boundary_y = float(tokens[14])
            vector_to_grain_boundary_z = float(tokens[15])
            magnitude_of_vector_to_grain_boundary = tokens[16]
            unit_vector_to_grain_boundary_x = tokens[17]
            unit_vector_to_grain_boundary_y = tokens[18]
            unit_vector_to_grain_boundary_z = tokens[19]
            
            phi1 = tokens[20]
            Phi = tokens[21]
            phi2 = tokens[22]
            nearest_grain_phi1 = tokens[23]
            nearest_grain_Phi = tokens[24]
            nearest_grain_phi2 = tokens[25]
            misorientation = tokens[26]
            
            normal_x = tokens[27]
            normal_y = tokens[28]
            normal_z = tokens[29]
            raw_angle = tokens[30]
            scaled_angle = tokens[31].strip()

            opposite_point = abs(x - reference_x)
            adjacent_point = abs(y - reference_y)

            # If the line isn't exactly vertical (i.e.: does not have undefined slope)
            if adjacent_point != 0:
                calculated_theta = atan(opposite_point / adjacent_point)

                # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
                if y > reference_y:
                    calculated_theta = pi - calculated_theta
            else:
                calculated_theta = pi / 2

            # CALCULATE THE DISTANCE FROM REFERENCE TO POINTS
            current_distance = ((x - reference_x)**2 + (y - reference_y)**2)**.5

            if calculated_theta < pi / 2:
                forward_y = y - step_size * cos(calculated_theta)
                backward_y = y + step_size * cos(calculated_theta)
                if x >= reference_x:
                    forward_x = x + step_size * sin(calculated_theta)
                    backward_x = x - step_size * sin(calculated_theta)
                else:
                    forward_x = x - step_size * sin(calculated_theta)
                    backward_x = x + step_size * sin(calculated_theta)
            else:
                temp_theta = pi - calculated_theta
                forward_y = y + step_size * cos(temp_theta)
                backward_y = y - step_size * cos(temp_theta)
                if x >= reference_x:
                    forward_x = x + step_size * sin(calculated_theta)
                    backward_x = x - step_size * sin(calculated_theta)
                else:
                    forward_x = x - step_size * sin(calculated_theta)
                    backward_x = x + step_size * sin(calculated_theta)

            if backward_x <= reference_x and current_distance < 10: # If we went too far back
                backward_x = reference_x
                backward_y = reference_y

                # FIND THE Z FROM A KD TREE
            forward_index = kd_tree.query((forward_x, forward_y))[1]
            backward_index = kd_tree.query((backward_x, backward_y))[1]
            forward_z = crack_surface_points[forward_index][2]
            backward_z = crack_surface_points[backward_index][2]

            first_vector_x = (x - backward_x) * 2
            first_vector_y = (y - backward_y) * 2
            first_vector_z = (z - backward_z) * 2
            first_vector_magnitude = (first_vector_x**2 + first_vector_y**2 + first_vector_z**2)**.5

            second_vector_x = (forward_x - x)
            second_vector_y = (forward_y - y)
            second_vector_z = (forward_z - z)
            second_vector_magnitude = (second_vector_x**2 + second_vector_y**2 + second_vector_z**2)**.5

            dot = first_vector_x*second_vector_x + first_vector_y*second_vector_y + first_vector_z*second_vector_z
            mags = first_vector_magnitude * second_vector_magnitude
            first_vector = (first_vector_x, first_vector_y, first_vector_z)
            second_vector = (second_vector_x, second_vector_y, second_vector_z)
            beta = angle_between(first_vector, second_vector)
            change_in_z = second_vector_z - first_vector_z

            # Write it to the file
            to_write = '{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'\
                                .format(dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,
                                grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,
                                nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,
                                vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,
                                magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,
                                unit_vector_to_grain_boundary_y, unit_vector_to_grain_boundary_z, phi1, Phi, phi2, 
                                nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2, misorientation, 
                                normal_x, normal_y, normal_z, raw_angle, scaled_angle)
            w.write(to_write)
print('done')